## Reading the feature content raw data


The `simple_data` and the `grouped_data` have the same shape. 

```
simple_data = [ [] , [] , ... ]
grouped_data = [ [] , [] , ... ]
```

For both `simple_data` and `grouped_data`, each element is a policy.
Each policy is a list, in which, each element in a tuple.

```
policy = [ (policy_header, policy_name, article_title, raw_content) , (...) , ... ]
```

In [2]:
import pickle, os
from utils.config import FEATURES_PATH

simple_results = os.path.join(FEATURES_PATH, 'simple_files_results.pkl')
grouped_results = os.path.join(FEATURES_PATH, 'grouped_files_results.pkl')


with open(simple_results, 'rb') as f:
    simple_data = pickle.load(f)
    
with open(grouped_results, 'rb') as f:
    grouped_data = pickle.load(f)

In [3]:
all_data = simple_data + grouped_data # we can sum list and we add the elements at the end of the first list

for policy in all_data:
    for article in policy:
        policy_header = article[0]
        policy_name = article[1]
        article_title = article[2]
        raw_content = article[3]
        break # we only want to see the first policy
    break

print(f"Policy header: {policy_header}")
print(f"Policy name: {policy_name}")
print(f"Article title: {article_title}")
print(f"Raw content: {raw_content[:100]}...")

Policy header: PÓLIZA DE ACCIDENTES PERSONALES REEMBOLSO GASTOS MÉDICOS
Policy name: POL120190177
Article title: ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO
Raw content: Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes...


We can obtain a list of all the contents of the policies by doing the following:

In [4]:
# List comprehension 
all_contents = [article[3] for policy in all_data for article in policy]

# For loop
all_contents = []
for policy in all_data:
    for article in policy:
        all_contents.append(article[3])

In [13]:
# 213 articulos en total
len(all_contents)

213

In [5]:
all_contents[0]

'Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las\nnormas legales de carácter imperativo establecidas en el título VIII, del Libro II, del Código de Comercio. Sin\nembargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el\nasegurado o el beneficiario.\n'

In [6]:
# We should remove the '\n' and '\t' characters, and the extra spaces

import re

# Remove '\n' and '\t' characters
all_contents = [re.sub(r'[\n\t]', ' ', content) for content in all_contents]
    
# Remove extra spaces
all_contents = [re.sub(r'\s+', ' ', content) for content in all_contents]

In [7]:
all_contents[0]

'Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las normas legales de carácter imperativo establecidas en el título VIII, del Libro II, del Código de Comercio. Sin embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el asegurado o el beneficiario. '

In [ ]:
# now we can preprocess the text

## Preprocessing of the text

#### (First part of the G_preprocess_results.py)

Remember to install the model locally:

```python -m spacy download es_core_news_sm```

In [8]:
from eda_tools.preprocess_stopwords import apply_stopwords, preprocess_non_printable_characters

preprocess_non_printable_characters(all_contents[0])

'se aplicaran al presente contrato de seguro las disposiciones contenidas en los articulos siguientes y las normas legales de caracter imperativo establecidas en el titulo viii, del libro ii, del codigo de comercio. sin embargo, se entenderan validas las estipulaciones contractuales que sean mas beneficiosas para el asegurado o el beneficiario.'

In [14]:
preprocessing = [preprocess_non_printable_characters(content) for content in all_contents]

In [18]:
features = apply_stopwords(preprocessing)

100%|██████████| 213/213 [00:08<00:00, 25.81it/s]


In [20]:
features[0]

'aplicaran presente contrato seguro disposiciones contenidas articulos siguientes normas legales caracter imperativo establecidas titulo viii , libro ii , codigo comercio . , entenderan validas estipulaciones contractuales beneficiosas asegurado beneficiario .'

In [22]:
# comparacion final: para el articulo 1 poliza 1 = reducción del 25%

# incluso podría dividirse en chunks y luego aplicar u

len(all_contents[0]), len(features[0]), round(len(features[0])/len(all_contents[0])*100, 1)

(346, 259, 74.9)

## Generating Langchain Document

#### (Second part of the G_preprocess_results.py)

In [26]:
from langchain_core.documents.base import Document

feature_contents = []
ci = 0
for policy in all_data:
    for article in policy:
        if "_" in article[1]:
            policy_file = article[1].split("_")[0]
            policy_name = article[1].split("_")[1]
        else:
            policy_file = article[1]
            policy_name = article[1]

        policy_header = article[0]
        article_title = article[2]
        page_content = features[ci]
        ci += 1

        feature_contents.append(
            Document(
                page_content=page_content,
                metadata={
                    "policy_file": policy_file + ".pdf",
                    "policy_name": policy_name,
                    "policy_header": policy_header,
                    "article_title": article_title
                }
            )
        )

In [28]:
feature_contents[0]

Document(page_content='aplicaran presente contrato seguro disposiciones contenidas articulos siguientes normas legales caracter imperativo establecidas titulo viii , libro ii , codigo comercio . , entenderan validas estipulaciones contractuales beneficiosas asegurado beneficiario .', metadata={'policy_file': 'POL120190177.pdf', 'policy_name': 'POL120190177', 'policy_header': 'PÓLIZA DE ACCIDENTES PERSONALES REEMBOLSO GASTOS MÉDICOS', 'article_title': 'ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO'})

In [29]:
feature_contents[-1]

Document(page_content='efectos contrato seguro entendera : 1 . contratante : persona natural juridica suscribe contrato compañia seguros asume obligaciones deriven , disposiciones contrato naturaleza cumplidas asegurado . 2 . asegurado titular : titular poliza , individualizado condiciones particulares poliza . asegurado titular contratante poliza detalle contrario condiciones particulares poliza . 3 . asegurados dependientes : asegurados dependientes integrantes grupo familiar titular especificados condiciones particulares poliza . asegurados deberan cumplir requisitos edad asegurabilidad establecidos condiciones particulares poliza encontrarse señalados tales respectivas condiciones particulares poliza . categorias asegurados dependientes consten condiciones particulares , podran contemplar condiciones requisitos excluyentes . condiciones generales utilice termino " asegurado " , indicar asegurado titular , debera entender termino abarca comprende asegurado titular demas asegurados .

In [ ]:
# guardar en formato pickle dentro de la carpeta `.serialized_features/`

## Creating Vector Database Collections for the feature content

#### H_vdb_create.py 

In [ ]:
# guardar los vectores en la vdb

----

# old stuff

## reading the article features

In [7]:
# feature files and feature articles
# single and grouped

import pickle

from utils.config import FEATURES_PATH


with open(f"{FEATURES_PATH}/feature_files.pkl", "rb") as f:
    feature_files = pickle.load(f)
with open(f"{FEATURES_PATH}/feature_articles.pkl", "rb") as f:
    feature_articles = pickle.load(f)
with open(f"{FEATURES_PATH}/grouped_feature_files.pkl", "rb") as f:
    grouped_feature_files = pickle.load(f)
with open(f"{FEATURES_PATH}/grouped_feature_articles.pkl", "rb") as f:
    grouped_feature_articles = pickle.load(f)


In [2]:
feature_files

[Document(page_content='póliza de accidentes personales reembolso gastos médicos reglas aplicables al contrato cobertura y materia asegurada limitaciones de las coberturas definiciones exclusiones obligaciones del asegurado declaraciones del contratante y del asegurado primas y efecto del no pago de la prima denuncia de siniestro calculo de los gastos reembolsables liquidación de los gastos, forma de pago y monto máximo de gastos aplicación del deducible vigencia y duración del contrato de seguro incorporación de asegurados e inicio de la vigencia de la cobertura terminación del contrato ajuste de la prima moneda o unidad del contrato rehabilitación de la póliza contribuciones e impuestos comunicación entre las partes solución de controversias cláusulas adicionales domicilio', metadata={'file': 'POL120190177'}),
 Document(page_content='seguro colectivo complementario de salud reglas aplicables al contrato cobertura definiciones monto máximo de reembolso duplicación de beneficios exclus

In [6]:
grouped_feature_files

[Document(page_content='seguro para prestaciones médicas de alto costo reglas aplicables al contrato cobertura definiciones exclusiones carencia obligaciones del asegurado declaraciones del asegurado primas y efectos del no pago de las primas beneficiarios denuncia de siniestros vigencia y terminación comunicación entre las partes solución de controversias clausulas adicionales domicilio', metadata={'file': 'POL320190074/POL320190074'}),
 Document(page_content='exoneración de pago de primas por fallecimiento del asegurado titular cobertura exclusiones carencia beneficiarios denuncia de siniestro cláusulas aplicables', metadata={'file': 'POL320190074/CAD220130244'}),
 Document(page_content='seguro colectivo complementario de salud reglas aplicables al contrato cobertura definiciones monto máximo de reembolso duplicación de beneficios exclusiones riesgos cubiertos bajo estipulación expresa obligaciones del asegurado declaraciones del asegurado beneficiarios primas y efecto del no pago de

In [9]:
len(feature_articles)

6

In [10]:
print(feature_articles[0][0].metadata)
print(feature_articles[0][-1].metadata)

{'file': 'POL120190177', 'article': 'ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO'}
{'file': 'POL120190177', 'article': 'ARTÍCULO 23°: DOMICILIO'}


In [13]:
# feature_grouped_articles
print(grouped_feature_articles[-1][0].metadata)
print(grouped_feature_articles[-2][-1].metadata)

{'file': 'POL320200071/POL320160108', 'article': 'ARTÍCULO 1. REGLAS APLICABLES AL CONTRATO. '}
{'file': 'POL320200071/POL320200071', 'article': 'ARTÍCULO 20°. CLÁUSULAS ADICIONALES.-'}
